In [3]:
!pip install -U sentence-transformers

In [4]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import os,torch,glob,pickle,zipfile

In [5]:
img_model = SentenceTransformer("clip-ViT-B-32")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from tqdm.autonotebook import tqdm
img_folder = 'photos/'
if not os.path.exists(img_folder) or len(os.listdir(img_folder)) == 0:
    os.makedirs(img_folder, exist_ok=True)

    photo_filename = 'unsplash-25k-photos.zip'
    if not os.path.exists(photo_filename):   #Download dataset if does not exist
        util.http_get('http://sbert.net/datasets/'+photo_filename, photo_filename)

    #Extract all images
    with zipfile.ZipFile(photo_filename, 'r') as zf:
        for member in tqdm(zf.infolist(), desc='Extracting'):
            zf.extract(member, img_folder)

  0%|          | 0.00/1.92G [00:00<?, ?B/s]

Extracting:   0%|          | 0/24996 [00:00<?, ?it/s]

In [6]:
images_names = list(glob.glob(f'photos/*.jpg'))
print('Images',len(images_names))


Images 24996


In [ ]:
img_emb = img_model.encode([Image.open(filepath) for filepath in images_names], batch_size=128, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/196 [00:00<?, ?it/s]

In [1]:
from IPython.display import display
from IPython.display import Image as IPImage
def search(query, k=3):
    # First, we encode the query (which can either be an image or a text string)
    query_emb = img_model.encode([query], convert_to_tensor=True, show_progress_bar=False)

    # Then, we use the util.semantic_search function, which computes the cosine-similarity
    # between the query embedding and all image embeddings.
    # It then returns the top_k highest ranked images, which we output
    hits = util.semantic_search(query_emb, img_emb, top_k=k)[0]

    print("Query:")
    display(query)
    for hit in hits:
        print(images_names[hit['corpus_id']])
        display(IPImage(os.path.join(img_folder, images_names[hit['corpus_id']]), width=200))

In [2]:
search("Two dogs playing in the snow")

NameError: name 'img_model' is not defined